In [ ]:
import json,cv2,random
random.seed(42)

f = open('simplified_annotations.json')
data = json.load(f)

#for future usage. 
test_list = ['001_6(18)_Ndef_1fl.JPG','006_5(17)_Ndef_1fl.JPG','011_2(14)_Ndef_1fl.JPG',
             '016_1(13)_Ndef_2fl.JPG','021_4(16)_Ndef_2fl.JPG','026_6(18)_Ndef_2fl.JPG',
             '031_6(18)_Ndef_3fl.JPG','036_5(17)_Ndef_3fl.JPG']

#cropped image width and height
cw,ch = 200,200

#Label studio transforms the bounding box. So here only transforming it back
#to get the pixel value

def ls_to_px_int(x,y,w,h):
    return int(x*40),int(y*60),int(w*40),int(h*60)

#Checking if a particular point lies inside a rectangle given x,y,w and h
def chk_points(cx,cy,cw,ch,x,y):
    if (cx<x<cx+cw) and(cy<y<cy+ch):
        return True
    return False

In [ ]:
#assuming the first image of v8 only. 
#string manipulation for the filename 
fn = data[0]['image'][24:]
img = cv2.imread('v8/' + fn[:5]+'('+ fn[5:7]+')'+fn[-13:])


#n = the number of cropped image to create
#check_flag = True if all points of the bounding box lies inside the cropped image
count = 0
n = 10
while count<n:
    check_flag = False
    #select a random starting point for cropped image
    (cx,cy) = (random.randint(0,6000-ch),random.randint(0,4000-cw)) #original dimension 4000x6000. But for opencv it is reversed.

    #check all the bounding box for if any bounding box lies inside the cropped image
    #if no such bounding box exists then randomly pick a new starting point
    for i in range(len(data[0]['label'])):
        x,y,w,h,_,_,_,_ = data[0]['label'][i].values()
        px,py,pw,ph = ls_to_px_int(x,y,w,h)
        
        a = chk_points(cx,cy,cw,ch,py,px)
        b = chk_points(cx,cy,cw,ch,py+ph,px)
        c = chk_points(cx,cy,cw,ch,py,px+pw)
        d = chk_points(cx,cy,cw,ch,py+ph,px+pw)
        
        check_flag = a and d and c and d
        
        if check_flag == True:
            print(cx,cy)
            crop_image = img[cx:cx+cw, cy:cy+ch]
            cv2.imwrite("data_store/"+fn+str(count)+".jpg", crop_image)
            count +=1
            break